# MHKDR APIs Comparison

The objective of this notebook is to enumerate the variables contained in both the first and second MHKDR api, in order to decide how to include MHKDR data into the package without loss of data or duplication.

The fear is that both contain a subset of each other. In such a situation, the use of one but not both APIs would cause data loss. 

This notebook is considered complete when it has answered the questions:
1. Is there data contained in API 1 that is NOT present in API 2?
2. Is there data contained in API 2 that is NOT present in API 1 - We know "yes"
3. What is the cause of mismatch of the number of records in each API?

### Setup

In [1]:
import requests
import json
import pandas as pd
import re

mhkdr_api_1 = 'https://mhkdr.openei.org/api?action=getSubmissionsForPRIMRE'
mhkdr_api_2 = 'https://mhkdr.openei.org/data.json'

In [2]:
mhkdr_1_response = requests.get(mhkdr_api_1)
mhkdr_2_response = requests.get(mhkdr_api_2)

In [3]:
mhkdr_1_response_json = mhkdr_1_response.json()
mhkdr_2_response_json = mhkdr_2_response.json()

### Dev

#### 1. & 2. - Data contained in each MHKDR API

In [4]:
mhkdr_1_df = pd.DataFrame(mhkdr_1_response_json)
mhkdr_2_df = pd.DataFrame(mhkdr_2_response_json['dataset'])

In [5]:
a = len(mhkdr_1_df)
b = len(mhkdr_2_df)
print(f'API 1 Number of MHKDR Entries : {a}\nAPI 2 Number of MHKDR Entries : {b}')

API 1 Number of MHKDR Entries : 400
API 2 Number of MHKDR Entries : 335


In [6]:
a = len(mhkdr_1_response_json[0])
b = len(mhkdr_2_response_json['dataset'][0])
print(f'API 1 Number of MHKDR Columns : {a}\nAPI 2 Number of MHKDR Columns : {b}')

API 1 Number of MHKDR Columns : 14
API 2 Number of MHKDR Columns : 20


14 fields in api 1 vs 20 fields in api 2

##### Comparison Preamble

The following table compiles the different variables included in the two MHKDR APIs. The intent is to use the table as a markdown convention to convey a venn diagram, using the variable names to indicate the inclusion of that data in either or both APIs. This structure is problematic because there are cases where:
1. two variables have different names but have similar contents
2. two variables have similar names but have different contents

Also, because the table format inherently includes ordering, it is unclear wheter the variable names should be used with this ordering to form an index, where similar-named things are compared. The competing approach would be to have similar contents requiring an entire row. This way, there is no possibility for two columns of the same row to be filled (as the contents being similar in both indicates that the "both" column should be populated instead).

I will proceed with the second method: each row will pertain to one nugget of data. After following API 1 through all of its fields, recording similar and different fields accordingly, I will proceed across all fields of API 2, likewise recording similar and different fields. A "nugget" is here defined as an atomic datum that is not interchangable with another datum from that or the other API without incurring data loss. We work from the axiom that datum interchangability indicates field (column) interchangability.


|1|both|2|
|:---:|:---:|:---:|
||(URI, landingPage, sourceURL) / (identifier, landingPage)||
|type|||
||(title) / (title)||
||(description) / (description)||
|author|||
|organization|||
|originationDate|||
||(spatial) / (spatial)?||
|technologyType|||
||(tags) / (keyword)||
|signatureProject|||
||(modifiedDate) / (modified)?||
|||@type|
|||accessLevel|
|||bureauCode|
|||license|
|||issued|
|||dataQuality|
|||projectTitle|
|||projectNumber|
|||publisher|
|||contactPoint|
|||programCode|
|||distribution|
|||DOI|

In [7]:
mhkdr_1_response_json[0].keys()

dict_keys(['URI', 'type', 'landingPage', 'sourceURL', 'title', 'description', 'author', 'organization', 'originationDate', 'spatial', 'technologyType', 'tags', 'signatureProject', 'modifiedDate'])

In [8]:
print(mhkdr_2_response_json['dataset'][0].keys())

dict_keys(['@type', 'identifier', 'accessLevel', 'bureauCode', 'license', 'issued', 'dataQuality', 'title', 'description', 'keyword', 'projectLead', 'projectTitle', 'projectNumber', 'modified', 'publisher', 'contactPoint', 'programCode', 'landingPage', 'distribution', 'spatial'])


In [9]:
mhkdr_1_df.tail(1)

,URI,type,landingPage,sourceURL,title,description,author,organization,originationDate,spatial,technologyType,tags,signatureProject,modifiedDate
399,https://mhkdr.openei.org/submissions/1,"[Dataset, Dataset/OnlineTool, Document/Report]",https://mhkdr.openei.org/submissions/1,https://mhkdr.openei.org/submissions/1,MHKDR Data Management and Best Practices for S...,Resources for MHKDR data submitters and curato...,"[Jon Weers, Nicole Taverna, Jay Huggins, RJ Sc...",[National Renewable Energy Laboratory],2021-12-15 07:00:00,"{'boundingCoordinatesNE': [83, 180], 'bounding...",[],"[MHK, Marine, Hydrokinetic, energy, power, dat...",[],2022-05-26 18:08:40


In [10]:
mhkdr_2_df.head(1)

,@type,identifier,accessLevel,bureauCode,license,issued,dataQuality,title,description,keyword,...,projectTitle,projectNumber,modified,publisher,contactPoint,programCode,landingPage,distribution,spatial,DOI
0,dcat:Dataset,https://mhkdr.openei.org/submissions/1,public,[019:20],https://creativecommons.org/licenses/by/4.0/,2021-12-15T07:00:00Z,True,MHKDR Data Management and Best Practices for S...,Resources for MHKDR data submitters and curato...,"[MHK, Marine, Hydrokinetic, energy, power, dat...",...,Marine and Hydrokinetic Data Repository (MHKDR),35007,2022-05-26T18:08:40Z,"{'@type': 'org:Organization', 'name': 'RJ Scavo'}","{'@type': 'vcard:Contact', 'fn': 'MHKDR Help',...",[019:009],https://mhkdr.openei.org/submissions/1,"[{'@type': 'dcat:Distribution', 'description':...","{""type"":""Polygon"",""coordinates"":[[[-180,-83],[...",NaN


#### Full Outputs

In [11]:
# Raw json api 1 output  -  this entry corresponds to the entry of api 2 below
mhkdr_1_response_json[399]

{'URI': 'https://mhkdr.openei.org/submissions/1',
 'type': ['Dataset', 'Dataset/OnlineTool', 'Document/Report'],
 'landingPage': 'https://mhkdr.openei.org/submissions/1',
 'sourceURL': 'https://mhkdr.openei.org/submissions/1',
 'title': 'MHKDR Data Management and Best Practices for Submitters and Curators',
 'description': 'Resources for MHKDR data submitters and curators, including training videos, step-by-step guides on data submission, and detailed documentation of the MHKDR. The Data Management and Submission Best Practices document also contains API access and metadata schema information for developers interested in harvesting MHKDR metadata for federation or inclusion in their local catalogs.\n',
 'author': ['Jon Weers', 'Nicole Taverna', 'Jay Huggins', 'RJ Scavo'],
 'organization': ['National Renewable Energy Laboratory'],
 'originationDate': '2021-12-15 07:00:00',
 'spatial': {'boundingCoordinatesNE': [83, 180],
  'boundingCoordinatesSW': [-83, -180],
  'extent': 'boundingBox'}

In [12]:
# Raw json api 2 output
mhkdr_2_response_json['dataset'][0]

{'@type': 'dcat:Dataset',
 'identifier': 'https://mhkdr.openei.org/submissions/1',
 'accessLevel': 'public',
 'bureauCode': ['019:20'],
 'license': 'https://creativecommons.org/licenses/by/4.0/',
 'issued': '2021-12-15T07:00:00Z',
 'dataQuality': True,
 'title': 'MHKDR Data Management and Best Practices for Submitters and Curators',
 'description': 'Resources for MHKDR data submitters and curators, including training videos, step-by-step guides on data submission, and detailed documentation of the MHKDR. The Data Management and Submission Best Practices document also contains API access and metadata schema information for developers interested in harvesting MHKDR metadata for federation or inclusion in their local catalogs.\n',
 'keyword': ['MHK',
  'Marine',
  'Hydrokinetic',
  'energy',
  'power',
  'data',
  'MHKDR',
  'federation',
  'metadata',
  'standards',
  'submission',
  'training',
  'best practices',
  'guide',
  'API',
  'management',
  'storage'],
 'projectLead': 'Bill M

#### 1. & 2. - Data Contained in each MHKDR API Conclusions

This notebook is considered complete when it has answered the questions:
1. Is there data contained in API 1 that is NOT present in API 2? - Yes!
2. Is there data contained in API 2 that is NOT present in API 1? - Yes!

#### 3. What is the cause of mismatch of the number of records in each API?


To Find the answer to this question, I need to create a list of all the entries that are included in one API but not the other. Again, this might go both ways:
1. Are there entries present in API 1 but not API 2?
2. Are there entries present in API 2 but not API 1?

I must subsequently open these entries and attempt to determine the cause of the mismatch.

In [13]:
mhkdr_1_uri_lst = list(mhkdr_1_df['landingPage'])
mhkdr_2_uri_lst = list(mhkdr_2_df['landingPage'])

In [14]:
def find_entry_id(entry_uri):
    '''
    This function takes in the url of a MHKDR entry, and returns the entry_id of that page. 
    The 'entry_id' is the integer at the end of the url, which is unique to each MHKDR entry.
    The regex used in this function relies on the fact that the only number in the url is the id.
    '''
    rule = re.compile(r'\d+')
    matches_rule = rule.findall(entry_uri)
    entry_id = int(matches_rule[0])

    return entry_id

In [15]:
mhkdr_1_ids = list()
for i in mhkdr_1_uri_lst:
    id = find_entry_id(i)
    mhkdr_1_ids.append(id)

mhkdr_2_ids = list()
for i in mhkdr_2_uri_lst:
    id = find_entry_id(i)
    mhkdr_2_ids.append(id)

In [16]:
a = len(mhkdr_1_ids)
b = len(mhkdr_2_ids)
print(f'Number of entries returned by API 1 : {a}\nNumber of entries returned by API 2 : {b}')

Number of entries returned by API 1 : 400
Number of entries returned by API 2 : 335


Good. As expected, there is a mismatch in the number of entries returned by the two APIs. 

To get the set of entries that is included by one and not the other, I will append this id column to the original ("core") tables, and then do a merge (Anti-Join) so that only those rows are preserved whose values of 'id' do not match the other table. Also, I will create a column entitled 'api' so that the results of API 1 and API 2 are easily differentiable in the final table. This will make it easy to see how many mismatches are unique to API 1 vs unique to API 2.

#### Prep

In [17]:
mhkdr_1_df['id'] = mhkdr_1_ids
mhkdr_2_df['id'] = mhkdr_2_ids

In [18]:
mhkdr_1_df['api'] = 1
mhkdr_2_df['api'] = 2

In [19]:
mhkdr_1_df.head(1)

,URI,type,landingPage,sourceURL,title,description,author,organization,originationDate,spatial,technologyType,tags,signatureProject,modifiedDate,id,api
0,https://mhkdr.openei.org/submissions/548,"[Dataset, Document/Report]",https://mhkdr.openei.org/submissions/548,https://mhkdr.openei.org/submissions/548,"CalWave - xWave Device, Non-Commercially Sensi...",CalWave has developed a submerged pressure dif...,"[Marcus Lehmann, Ryan Davidson]",[CalWave Power Technologies Inc.],2024-02-29 07:00:00,"{'boundingCoordinatesNE': [44.63067800397145, ...",[Wave],"[MHK, Marine, Hydrokinetic, energy, power, wav...",[],2024-04-25 20:40:00,548,1


In [20]:
mhkdr_2_df.tail(1)

,@type,identifier,accessLevel,bureauCode,license,issued,dataQuality,title,description,keyword,...,modified,publisher,contactPoint,programCode,landingPage,distribution,spatial,DOI,id,api
334,dcat:Dataset,https://mhkdr.openei.org/submissions/548,public,[019:20],https://creativecommons.org/licenses/by/4.0/,2024-02-29T07:00:00Z,True,"CalWave - xWave Device, Non-Commercially Sensi...",CalWave has developed a submerged pressure dif...,"[MHK, Marine, Hydrokinetic, energy, power, wav...",...,2024-04-25T20:40:00Z,"{'@type': 'org:Organization', 'name': 'Ryan Da...","{'@type': 'vcard:Contact', 'fn': 'Marcus Lehma...",[019:009],https://mhkdr.openei.org/submissions/548,"[{'@type': 'dcat:Distribution', 'description':...","{""type"":""Polygon"",""coordinates"":[[[-125.910046...",NaN,548,2


#### Merge

In [21]:
mhkdr_diff_df = (mhkdr_1_df.merge(mhkdr_2_df, on='id', how='outer', indicator=True)
                     .query('_merge != "both"')
                     .drop('_merge', 1))

C:\Users\deca761\AppData\Local\Temp\ipykernel_15976\3406136988.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .drop('_merge', 1))


In [22]:
len(mhkdr_diff_df)

65

In [23]:
65 + 335

400

Sanity check! Indeed the merge worked.

If the merge was successful, we should have exactly those entries that are inlcuded in one, but not both, APIs. Therefore the number of entries included in the final table should equal the difference between the number of entries in the original two tables. As we can see, we have 65 entries returned to us, and, when added to the entries contained in API 2, we return the number of entries contained in API 1.

Now we will proceed to verify that there are only entries in this final table (mhkdr_diff_df) that originate from API 1 (mhkdr_1_df). If this is not the case, our prior sanity check does not hold, and this deserves further investigation.

In [24]:
mhkdr_diff_df.keys()

Index(['URI', 'type', 'landingPage_x', 'sourceURL', 'title_x', 'description_x',
       'author', 'organization', 'originationDate', 'spatial_x',
       'technologyType', 'tags', 'signatureProject', 'modifiedDate', 'id',
       'api_x', '@type', 'identifier', 'accessLevel', 'bureauCode', 'license',
       'issued', 'dataQuality', 'title_y', 'description_y', 'keyword',
       'projectLead', 'projectTitle', 'projectNumber', 'modified', 'publisher',
       'contactPoint', 'programCode', 'landingPage_y', 'distribution',
       'spatial_y', 'DOI', 'api_y'],
      dtype='object')

In [25]:
print(mhkdr_diff_df['api_x'].unique()[0])

1


In [26]:
print(mhkdr_diff_df['api_y'].unique()[0])

nan


Perfect! 

As we can see, although there was some befuddling in the merge, thereby creating a bunch of extra variables, it is not a problem because we can see above that there are indeed only entries originating from API 1 in this table. Because of this, we can be confident that API 1 includes everything in API 2,  but not the reverse. We can therefore discard all columns ending with '_y', or complete a second merge to replace this one in order to get cleaner results.

#### 3.1 Conclusion (What is the cause of mismatch of the number of records in each API?)

1. Are there entries present in API 1 but not API 2? - Yes!
2. Are there entries present in API 2 but not API 1? - No!

#### 3.2 - Find the reason for the API mismatch

In [27]:
pd.set_option('display.max_columns', 1000)

In [28]:
mhkdr_diff_df.head(1)

,URI,type,landingPage_x,sourceURL,title_x,description_x,author,organization,originationDate,spatial_x,technologyType,tags,signatureProject,modifiedDate,id,api_x,@type,identifier,accessLevel,bureauCode,license,issued,dataQuality,title_y,description_y,keyword,projectLead,projectTitle,projectNumber,modified,publisher,contactPoint,programCode,landingPage_y,distribution,spatial_y,DOI,api_y
1,https://mhkdr.openei.org/submissions/547,"[Dataset, Document/Report, Dataset/Data]",https://mhkdr.openei.org/submissions/547,https://mhkdr.openei.org/submissions/547,CalWave - Reports and Plans for xWave Device D...,CalWave has developed a submerged pressure dif...,"[Thomas Boerner, Nigel Kojimoto, Marcus Lehman...",[CalWave Power Technologies Inc.],2024-02-29 07:00:00,"{'boundingCoordinatesNE': [44.69319166311689, ...",[Wave],"[MHK, Marine, Hydrokinetic, energy, power, Wav...",[],2024-04-25 20:48:08,547,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It appears that all of these entries are on data moratorium.

This makes integration very easy. This will correspond directly to Tethys / Tethys E where the "attachment" field is missing/nan.

Printing all of the links so that I can open every page and verify this hypothesis.

In [29]:
# links = list(mhkdr_diff_df['URI'])
# for i in links:
#     print(i)

Indeed all 65 entries have their Resources section marked as available at a future date.

#### 3.2 Conclusion (What is the cause of mismatch of the number of records in each API?)

I am fairly confident that the cause of the mismatch between then number of records in each API is that API 1 includes all entries, regardless of the state of the "resources" section within the entry, whereas API 2 only includes those entries whose "resources" section is not in moratorium.